In [1]:
import os 
import re
import json
from groq import Groq
import datetime

In [2]:
os.chdir("..")

In [3]:
base_data_path = "../model_output_data"

if os.path.exists(base_data_path):
    print("It exists")
else:
    print("Does not exist")

It exists


In [4]:
folders = [
    os.path.join(base_data_path, f) # Gives me the path so I can run datetime on it and get the most recent folder 
    for f in os.listdir(base_data_path) # Grabs all the folders in it
    if os.path.isdir(os.path.join(base_data_path, f)) # Validates that the path actually exists
]


In [5]:
latest_folder = max(folders, key=os.path.getctime)
print("Latest folder:", latest_folder)


Latest folder: ../model_output_data\research_on_java


## Temporary List to store data 

In [6]:
messages = []

In [7]:
tasks_path = os.path.join(latest_folder, "tasks.json")

with open(tasks_path, "r") as f:
    tasks = json.load(f)

In [8]:
retriever_agent = Groq()

In [9]:
search_results = [
    {
        "title": "Java Programming Language Overview",
        "url": "https://example.com/java-overview",
        "snippet": "Java is a high level object oriented programming language developed by Sun Microsystems."
    },
    {
        "title": "Key Features of Java",
        "url": "https://example.com/java-features",
        "snippet": "Java provides platform independence, automatic memory management, and strong security."
    }
]


In [ ]:
retriever_system_template = """
You are a research retrieval agent. 
That takes in the tasks and then analyzes them and after analyzing you provide information in the following
form you take 


Your job:
Extract accurate facts from the provided search results.

Rules:

- Use ONLY the given search results
- Do NOT invent information
- Prefer reliable sources
- Keep summaries concise and factual
- Include the source URL for every item
- If results are insufficient, return an empty list

Required output schema:

<answer>
[
  {{
    "source": "...",
    "title": "...",
    "summary": "...",
    "key_points": ["...", "..."]
  }}
]
</answer>
"""


In [12]:
for task in tasks:

    task_description = task["description"]

    retriever_system_prompt = retriever_system_template.format(
        task_description=task_description
    )

    search_results_str = json.dumps(search_results, indent=2)

    messages = [
        {"role": "system", "content": retriever_system_prompt},
        {"role": "user", "content": f"Search results:\n{search_results_str}"}
    ]

    completion = retriever_agent.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        stream=False,
    )

    reply = completion.choices[0].message.content
    print(reply)


<answer>
[
  {
    "source": "https://example.com/java-overview",
    "title": "Java Programming Language Overview",
    "summary": "Java is a high level object oriented programming language developed by Sun Microsystems.",
    "key_points": []
  },
  {
    "source": "https://example.com/java-features",
    "title": "Key Features of Java",
    "summary": "Java provides platform independence, automatic memory management, and strong security.",
    "key_points": []
  }
]</answer>
<answer>
[
  {
    "source": "https://example.com/java-overview",
    "title": "Java Programming Language Overview",
    "summary": "Java is a high level object oriented programming language developed by Sun Microsystems.",
    "key_points": []
  },
  {
    "source": "https://example.com/java-features",
    "title": "Key Features of Java",
    "summary": "Java provides platform independence, automatic memory management, and strong security.",
    "key_points": []
  }
]
</answer>
<answer>
[
  {
    "source": "htt